# Create an InterActive Graph of NTRL Lab Analysis Results

In [805]:
# CODE BLOCK 1

#import libraries
import pandas as pd
import numpy as np

from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict

#Libraries for date and time
import datetime
#from datetime import datetime
from datetime import timedelta
#----
import re

import openpyxl

#Libraries for machine learning
#from sklearn.metrics import mean_squared_error

#Libraries for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

In [806]:
# CODE BLOCK 2

#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

In [807]:
# CODE BLOCK 3

#### Create a daterange for the entire shudown duration

#Create an array of datetime as range of datetime incremented hourly for the entire shutdown duration
l = [datetime.datetime(2020, 10, 22, 0, 0),
    datetime.datetime(2020, 10, 23, 0, 0),
    datetime.datetime(2020, 10, 24, 0, 0),
    datetime.datetime(2020, 10, 25, 0, 0),
    datetime.datetime(2020, 10, 26, 0, 0),
    datetime.datetime(2020, 10, 27, 0, 0),
    datetime.datetime(2020, 10, 28, 0, 0),
    datetime.datetime(2020, 10, 29, 0, 0),
    datetime.datetime(2020, 10, 30, 0, 0),
    datetime.datetime(2020, 10, 31, 0, 0),
    datetime.datetime(2020, 11, 1, 0, 0),
    datetime.datetime(2020, 11, 2, 0, 0),
    datetime.datetime(2020, 11, 3, 0, 0),
    datetime.datetime(2020, 11, 4, 0, 0),
    datetime.datetime(2020, 11, 5, 0, 0),
    datetime.datetime(2020, 11, 6, 0, 0),
    datetime.datetime(2020, 11, 7, 0, 0),
    datetime.datetime(2020, 11, 8, 0, 0),
    datetime.datetime(2020, 11, 9, 0, 0),
    datetime.datetime(2020, 11, 10, 0, 0),
    datetime.datetime(2020, 11, 11, 0, 0)]

hours = np.arange(0,24)

t = [[item.replace(hour=hour) for hour in hours] for item in l] # nice list comprehension. see this closely.
u = np.array(t)
Shutdown_date_range = u.reshape((504,1))

Shutdown_date_range_1x504 = u.reshape((1,504)) # u is from the daterange code block
#daterange code END--------------------


#PREPARE DF LIST FOR LATER
# prepare a list of columns for use in summing up progress per area    
columns_dezn = ['De-Zn Feedline', '105TK01', '105TK02', '105TK03', '105TK05',
                '105SP01/02', '105PU01ABCDEFG', '105FT01A', '105FT01B', '105FT01C',
                '105FT01D', '105FT01E', '105FT01F', '105TK13', '105TK08']

columns_MS = ['106TK01', 'MS FEEDLINE', '106VE01', '106VE02', '106VE03', 
              '106VE04', '106VE05', '106HX02', '106VB01AB', '106CP01AB', 
              '106TH01', '106FT02A', '106FT02B', '106TK08', '106TK04',
              '106TK05', '106TK18', '106TK14', '106SP02', 'MS Common Vent/ 106SP01',
              '106SR00/02\n106SP03/04', '106SR03', '106SR04', '106TK15/ NaOH header line']

columns_108_chem = [ '108TK01', '108TK04','24" SPT tie in', 'VSLC RHL Line', '112TK00',
                    '112TK01', '114HP/FD01', '114TK01', '114TK02', '114 Pumps', 
                    '115TK01','115TK02']

columns_111_ls = ['111 Belt Conveyors\n111VS01\n111CR01AB', '111ML01','111TK01', '111PU01/\n111CY01', '111TK02', 
                  '530CT31', '525NG01']

In [861]:
# CODE BLOCK 4

# store the Filepath of the shutdown schedule excel file
#shutdown_filepath = r'\\THGOBNA002\thpal\05 Production\260 Tech - Shutdown\Shutdown 202009 Major Shutdown\Process B\00 Schedule\2020 Major SD Schedule Rev 2.4.xlsx'
shutdown_filepath2 = r'C:\Users\v.t.flores\Documents\2020 Major SD Progress Manual Update.xlsx'

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
# Load the shutdown schedule excel file
raw_shutdown_sched = pd.read_excel(shutdown_filepath2, sheet_name='Sheet1', index_col=False)

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
# CLEAN UP THE 2020 Major SD Progress Manual Update File DF
raw_shutdown_sched = raw_shutdown_sched.drop(raw_shutdown_sched.index[0:2])
shutdown_sched_1 = raw_shutdown_sched[['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']].iloc[57:]  ## SELECT THE ROWS AND COLUMNS NEEDED
shutdown_sched_1.columns = ['Equipment', 'Activities','START_plan','FINISH_plan',"Duration_hrs_plan", 'START_actual','FINISH_actual',"Duration_hrs_ETA"] ## RENAME THE COLUMNS

shutdown_sched_1 = shutdown_sched_1.reset_index().drop('index', axis=1)
shutdown_sched_1['Equipment'] = shutdown_sched_1['Equipment'].fillna(method='ffill') ## FILL THE BLANK CELLS IN THE EQUIPMENT COLUMN USING THE FFILL METHOD.

index_list = shutdown_sched_1[pd.isna(shutdown_sched_1['START_plan'])].index # takes the index of the column 'START_plan' with Nan values and stores it in the variable index_list
shutdown_sched_2 = shutdown_sched_1.drop(index_list) # Drop the nan values found in the 'START_plan' column then store to a new df called shutdown_sched_2

shutdown_sched_2['DATETIME'] = shutdown_sched_2['START_plan'] ## CREATE A DATETIME COLUMN EQUAL TO THE START COLUMN.
shutdown_sched_3 = shutdown_sched_2.set_index('DATETIME') ## SET THE INDEX TO DATETIME. THIS TELLS US WHEN THE ACTIVITY IS SCHEDULED TO START.

nan_index_list = shutdown_sched_3[pd.isna(shutdown_sched_3['Duration_hrs_plan'])].index # find nan values in the 'Duration_hrs_plan' column
shutdown_sched_3 = shutdown_sched_3.drop(nan_index_list) # Drop the nan values found in the 'Duration_hrs_plan'

# this code block computes the %progress per activity and the necessary columns needed to compute the % progress_actual

shutdown_sched_3['Reference_time'] = np.where(shutdown_sched_3['FINISH_actual'].isnull(), datetime.datetime.now(), shutdown_sched_3['FINISH_actual']) # column 9
shutdown_sched_3['Reference_time'] = [parse(str(ref_time)) for ref_time in shutdown_sched_3['Reference_time']]

shutdown_sched_3['START_plan'] = [parse(str(timestart)) for timestart in shutdown_sched_3['START_plan']]

shutdown_sched_3['Duration_hrs_plan'] = [datetime.timedelta(hours=x) for x in shutdown_sched_3['Duration_hrs_plan']] # column 5 . If erro occurs about datetime.timedelta, use timedelta(hours)

shutdown_sched_3['hours_since_start'] = np.where(shutdown_sched_3['START_plan'].notnull(), 
                                                 shutdown_sched_3['Reference_time'] - shutdown_sched_3['START_plan'], 
                                                 0) #compute the hours since the start of the activity. column 10 

shutdown_sched_3['Duration_hrs_ETA'] = np.where(shutdown_sched_3['Duration_hrs_plan'] < shutdown_sched_3['hours_since_start'], 
                                                shutdown_sched_3['hours_since_start'] + datetime.timedelta(hours=1), 
                                                shutdown_sched_3['Duration_hrs_plan']) # column 11

reference_time_now = datetime.datetime.now()
shutdown_sched_3['Percent_progress'] = 100*shutdown_sched_3['hours_since_start']/shutdown_sched_3['Duration_hrs_ETA'] # column 12 onwards

# Reset the index. This will be used for changing the names of the activities column to make each activity unique
shutdown_sched_3 = shutdown_sched_3.reset_index().drop('DATETIME', axis=1).reset_index()

shutdown_sched_3['index'] = [str(x) for x in shutdown_sched_3['index']] # change the index column to type string! awesome!
shutdown_sched_3['Activities'] = shutdown_sched_3['Activities'] +" "+shutdown_sched_3['index'] #Alter the activities name by adding an index number at the end

In [862]:
# CODE BLOCK 5 
# RUN THIS CODE ONLY ONCE. JUST TO CREATE A BLANK DF. THIS BLANK DF WILL BE UPDATED WITH NEW DATA FROM ACTUAL INPUT ON ACTIVITY PROGRESS.

df_Activities_Ref_time_slice = shutdown_sched_3[['Activities','Reference_time','Percent_progress']]
df_Act_ref2 = df_Activities_Ref_time_slice
Reference_time_list = df_Act_ref2['Reference_time'].unique()

# CREATE A NEW DF WITH THE TWO COLUMNS activities AND equipment by slicing the shutdown_sched_3
#1
activities_Equipment_for_joining = shutdown_sched_3[['Activities', 'Equipment']].reset_index()
#2
blank_df_820X504 = np.zeros((len(shutdown_sched_4['Activities']), len(Shutdown_date_range_1x504.T)))

#3
#CREATE THE BLANK DATAFRAME TO CONTAIN ALL THE DATERANGE AND THE EQUIPMENT NAME. SET THE INDEX TO THE ACTIVITIES

df_Shutdown_date_range = pd.DataFrame(Shutdown_date_range)
df_blank_df_820X504 = pd.DataFrame(blank_df_820X504)
df_blank_df_820X504.columns = df_Shutdown_date_range[0] # Create the column names from the daterange spanning the entire shutdown duration
#join the series for 'Actvities' and 'Equpment' from the shutdown_sched_4 
#set the Activities column as the index of this df
#group the df by equipment using the groupby function and using the mean() as the aggregate function
#transpose the df to make the columns of datetime be the index.
df_blank_df_820X504_actvts_eq = activities_Equipment_for_joining.join(df_blank_df_820X504, how='outer').set_index('Activities')

ACTUAL_PROGRESS_UPDATE = df_blank_df_820X504_actvts_eq

In [875]:
df_blank_df_820X504_actvts_eq.to_excel(r'C:\Users\v.t.flores\Documents\df_blank_df_820X504_actvts_eq_MS.xlsx')

#### test code: loading the actual_progress excel file

In [858]:
df_raw = pd.read_excel(r'C:\Users\v.t.flores\Documents\ACTUAL_PROGRESS_UPDATE.xlsx')
ACTUAL_PROGRESS_UPDATE = df_raw.set_index('Activities')
#ACTUAL_PROGRESS_UPDATE = ACTUAL_PROGRESS_UPDATE.drop('index', axis=1)

In [863]:
ACTUAL_PROGRESS_UPDATE

,index,Equipment,2020-10-22 00:00:00,2020-10-22 01:00:00,2020-10-22 02:00:00,2020-10-22 03:00:00,2020-10-22 04:00:00,2020-10-22 05:00:00,2020-10-22 06:00:00,2020-10-22 07:00:00,...,2020-11-11 14:00:00,2020-11-11 15:00:00,2020-11-11 16:00:00,2020-11-11 17:00:00,2020-11-11 18:00:00,2020-11-11 19:00:00,2020-11-11 20:00:00,2020-11-11 21:00:00,2020-11-11 22:00:00,2020-11-11 23:00:00
Activities,,,,,,,,,,,,,,,,,,,,,
104PU01 Stop 0,0,De-Zn Feedline,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOTO Installation (Valves & Pump) 1,1,De-Zn Feedline,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Feedline level lowering/105TK01 Inerting 2,2,De-Zn Feedline,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Feed Line Blinding/105TK01 inlet 3,3,De-Zn Feedline,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Feed Line Draining 4,4,De-Zn Feedline,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Conditioning (Chemical dosing with heat load) 815,815,530CT31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525CP01AB PMS Activity (1 compressor/ day) 816,816,525NG01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KV valves inspection 817,817,525NG01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [864]:
# CODE BLOCK 6

df_pivot_list = []
for i in range(len(Reference_time_list)):
    df = df_Act_ref2[df_Act_ref2['Reference_time'] == Reference_time_list[i]]   ## get all the rows that match the same Reference_time
    df.columns = ['Activities','Reference_time', 'prcent_progress']
    df_i_pivot = df.pivot(columns='Reference_time', index='Activities', values='prcent_progress')

    for i in range(len(df_i_pivot)): #
        activity_name = df_i_pivot.iloc[i].name # this is the activity name
        ref_time = df_i_pivot.columns[0] # this is the column name
        progress_val = df_i_pivot.iloc[i]
        
        if ref_time in ACTUAL_PROGRESS_UPDATE.columns: #check if the ref time (same as the date) is in daterange
            ACTUAL_PROGRESS_UPDATE.at[activity_name, ref_time] = progress_val #Updating the ACTUAL_PROGRESS_UPDATE with the %progress value from the df_pivot
        else:
            pass

print('PROGRESS Updated {}'.format(datetime.datetime.now()))            
# ACTUAL_PROGRESS_UPDATE
ACTUAL_PROGRESS_UPDATE.to_excel(r'C:\Users\v.t.flores\Documents\ACTUAL_PROGRESS_UPDATE.xlsx')


PROGRESS Updated 2020-10-16 16:54:16.447458


#### TEST CODES. TROUBLESHOOTING  ACTUAL_PROGRESS_UPDATE

In [838]:
# ACTUAL_PROGRESS_UPDATE

# The actual_progress_update should display the latest datetime values only.
# The progress from the previous datetime should be carried over to the next datetime values.
# Limit the displayed values up to the latest available datetime in the reference time.


In [865]:
df_test_ = ACTUAL_PROGRESS_UPDATE.groupby('Equipment')[ACTUAL_PROGRESS_UPDATE.columns[2:]].mean().T    

In [866]:
for col in df_test_.columns:
     df_test_[col] = df_test_[col].replace(to_replace=0, method='ffill')
    

In [867]:
df_test_

Equipment,105FT01A,105FT01B,105FT01C,105FT01D,105FT01E,105FT01F,105PU01ABCDEFG,105SP01/02,105TK01,105TK02,...,114TK02,115TK01,115TK02,"24"" SPT tie in",525NG01,530CT31,De-Zn Feedline,MS Common Vent/ 106SP01,MS FEEDLINE,VSLC RHL Line
2020-10-22 00:00:00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.846154,4.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2020-10-22 01:00:00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.846154,4.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2020-10-22 02:00:00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.846154,4.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,5.555556,0.0,0.000000,0.0
2020-10-22 03:00:00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.846154,4.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,5.555556,0.0,0.000000,0.0
2020-10-22 04:00:00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.846154,4.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,5.555556,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-11 19:00:00,11.111111,12.5,11.111111,11.111111,11.111111,0.0,16.666667,14.285714,3.846154,4.347826,...,0.0,0.0,0.0,0.0,0.0,0.0,8.333333,0.0,14.285714,0.0
2020-11-11 20:00:00,11.111111,12.5,11.111111,11.111111,11.111111,0.0,16.666667,14.285714,3.846154,4.347826,...,0.0,0.0,0.0,0.0,0.0,0.0,8.333333,0.0,14.285714,0.0
2020-11-11 21:00:00,11.111111,12.5,11.111111,11.111111,11.111111,0.0,16.666667,14.285714,3.846154,4.347826,...,0.0,0.0,0.0,0.0,0.0,0.0,8.333333,0.0,14.285714,0.0
2020-11-11 22:00:00,11.111111,12.5,11.111111,11.111111,11.111111,0.0,16.666667,14.285714,3.846154,4.347826,...,0.0,0.0,0.0,0.0,0.0,0.0,8.333333,0.0,14.285714,0.0


In [868]:
shutdown_sched_3[shutdown_sched_3['Reference_time']==shutdown_sched_3['Reference_time'].max()].index

Int64Index([433, 445], dtype='int64')

In [869]:
df_equipment_Grouped = df_test_
latest_time_in_Ref_Time = shutdown_sched_3['Reference_time'].max() # get the latest time in the reference time column. limit the displayed rows upto this point only.
df_equipment_Grouped = df_equipment_Grouped.loc[:latest_time_in_Ref_Time] # Using the .loc function, select the rows up to the point where the daterange matches the latest datetime in the reference time.


In [870]:
#df_equipment_Grouped = ACTUAL_PROGRESS_UPDATE.groupby('Equipment')[ACTUAL_PROGRESS_UPDATE.columns[2:]].mean().T    

#### END OF TESTING ACTUAL_PROGRESS_UPDATE TROUBLESHOOTING

In [871]:
# CODE BLOCK 7


# Compute the average %progress per area
actual_MS_progress_plan_MAINTE = np.mean(df_equipment_Grouped[columns_MS], axis=1)
actual_DeZn_progress_plan_MAINTE = np.mean(df_equipment_Grouped[columns_dezn], axis=1)
actual_FNTRL_chem_progress_plan_MAINTE = np.mean(df_equipment_Grouped[columns_108_chem], axis=1)
actual_LS_CT_progress_plan_MAINTE = np.mean(df_equipment_Grouped[columns_111_ls], axis=1)

# Create a column for the %progress values per area
df_equipment_Grouped['MS_progress_actual'] = actual_MS_progress_plan_MAINTE
df_equipment_Grouped['DeZn_progress_actual'] = actual_DeZn_progress_plan_MAINTE
df_equipment_Grouped['FNTRL_chem_progress_actual'] = actual_FNTRL_chem_progress_plan_MAINTE
df_equipment_Grouped['LS_CT_progress_actual'] = actual_LS_CT_progress_plan_MAINTE



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [878]:
df_raw_PLANNED_SCHED_PROGRESS = pd.read_excel(r'C:\Users\v.t.flores\Documents\Shutdown202010_tracker_data_planned.xlsx').set_index('DATETIME')
df_PLANNED_SCHED_PROGRESS = df_raw_PLANNED_SCHED_PROGRESS

In [881]:

# Compute the average %progress per area
MS_progress_plan_MAINTE = np.mean(df_PLANNED_SCHED_PROGRESS[columns_MS], axis=1)
DeZn_progress_plan_MAINTE = np.mean(df_PLANNED_SCHED_PROGRESS[columns_dezn], axis=1)
FNTRL_chem_progress_plan_MAINTE = np.mean(df_PLANNED_SCHED_PROGRESS[columns_108_chem], axis=1)
LS_CT_progress_plan_MAINTE = np.mean(df_PLANNED_SCHED_PROGRESS[columns_111_ls], axis=1)

# Create a column for the %progress values per area
df_PLANNED_SCHED_PROGRESS['MS_progress_PLAN'] = MS_progress_plan_MAINTE
df_PLANNED_SCHED_PROGRESS['DeZn_progress_PLAN'] = DeZn_progress_plan_MAINTE
df_PLANNED_SCHED_PROGRESS['FNTRL_chem_progress_PLAN'] = FNTRL_chem_progress_plan_MAINTE
df_PLANNED_SCHED_PROGRESS['LS_CT_progress_PLAN'] = LS_CT_progress_plan_MAINTE

#### Create the Plot

In [902]:
#### Plotting the SD progress including the maintenance activities
#
fig_SD_MAINTE_actual = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "scatter"}]])
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_equipment_Grouped['MS_progress_actual'].index, y=df_equipment_Grouped['MS_progress_actual'], name='MS_actual_progress',
                                         line=dict(color='#1f77b4', width=2),
                                         ), secondary_y=False,)
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_equipment_Grouped['DeZn_progress_actual'].index, y=df_equipment_Grouped['DeZn_progress_actual'], name='DeZn_actual_progress',
                                          line=dict(color='#ff7f0e', width=2),
                                         ), secondary_y=False,)     
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_equipment_Grouped['FNTRL_chem_progress_actual'].index, y=df_equipment_Grouped['FNTRL_chem_progress_actual'], name='FNTRL_chem_actual_progress',
                                          line=dict(color='#2ca02c', width=2),
                                         ), secondary_y=False,)    
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_equipment_Grouped['LS_CT_progress_actual'].index, y=df_equipment_Grouped['LS_CT_progress_actual'], name='LS_CT_actual_progress',
                                          line=dict(color='#d62728', width=2),
                                         ), secondary_y=False,)    

#-----------------------------------PLANNED SCHED DATA--------------------------------------------------------------------------------------------------------------------------------
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_PLANNED_SCHED_PROGRESS['MS_progress_PLAN'].index, y=df_PLANNED_SCHED_PROGRESS['MS_progress_PLAN'], name='MS_PLAN_progress',
                                          line=dict(color='#1f77b4', width=2, dash='dot')
                                         ), secondary_y=False,)
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_PLANNED_SCHED_PROGRESS['DeZn_progress_PLAN'].index, y=df_PLANNED_SCHED_PROGRESS['DeZn_progress_PLAN'], name='DeZn_PLAN_progress',
                                          line=dict(color='#ff7f0e', width=2, dash='dot',)
                                         ), secondary_y=False,)     
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_PLANNED_SCHED_PROGRESS['FNTRL_chem_progress_PLAN'].index, y=df_PLANNED_SCHED_PROGRESS['FNTRL_chem_progress_PLAN'], name='FNTRL_chem_PLAN_progress',
                                          line=dict(color='#2ca02c', width=2, dash='dot')
                                         ), secondary_y=False,)    
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_PLANNED_SCHED_PROGRESS['LS_CT_progress_PLAN'].index, y=df_PLANNED_SCHED_PROGRESS['LS_CT_progress_PLAN'], name='LS_CT_PLAN_progress',
                                          line=dict(color='#d62728', width=2, dash='dot')
                                         ), secondary_y=False,)    
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
############################################################################
'''
fig.add_trace(go.Scatter(x=month, y=high_2014, name='High 2014',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=month, y=low_2014, name = 'Low 2014',
                         line=dict(color='royalblue', width=4)))
fig.add_trace(go.Scatter(x=month, y=high_2007, name='High 2007',
                         line=dict(color='firebrick', width=4,
                              dash='dash') # dash options include 'dash', 'dot', and 'dashdot'
))
fig.add_trace(go.Scatter(x=month, y=low_2007, name='Low 2007',
                         line = dict(color='royalblue', width=4, dash='dash')))
fig.add_trace(go.Scatter(x=month, y=high_2000, name='High 2000',
                         line = dict(color='firebrick', width=4, dash='dot')))
fig.add_trace(go.Scatter(x=month, y=low_2000, name='Low 2000',
                         line=dict(color='royalblue', width=4, dash='dot')))
'''

############################################################################



#------------------
fig_SD_MAINTE_actual.update_xaxes(title_text="Datetime") # Set x-axis title
fig_SD_MAINTE_actual.update_yaxes(title_text="ShutDown Progress %",secondary_y=False) # Set y-axes titles primary axis

title_plot_SD = "<b>MS PLANT Shutdown Progress</b> Hourly Tracker"
fig_SD_MAINTE_actual.update_layout(title_text=title_plot_SD ,hovermode="x",
                       hoverdistance=100, # Distance to show hover label of data point
                       spikedistance=1000, # Distance to show spike
                       xaxis=dict(linecolor="#BCCCDC",
                                   showspikes=True, # Show spike line for X-axis
                                   # Format spike
                                   spikethickness=2,
                                   spikedash="dot",
                                   spikecolor="#999999",
                                   spikemode="across"), font=dict(size=10)
                               )

# Mark the datetime of power limitation by shading these regions with light color
fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-01 04:30:00',
            y0=0,
            x1='2020-11-01 19:00:00',
            y1=100,
            fillcolor="pink",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-08 16:00:00',
            y0=0,
            x1='2020-11-09 18:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )

fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-10-23 14:00:00',
            y0=0,
            x1='2020-10-25 08:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )


fig_SD_MAINTE_actual.update_shapes(dict(xref='x', yref='y'))

# Mark the datetime of power limitation by shading these regions with light color
fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-01 04:30:00',
            y0=0,
            x1='2020-11-01 19:00:00',
            y1=100,
            fillcolor="pink",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-08 16:00:00',
            y0=0,
            x1='2020-11-09 18:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )

fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-10-23 14:00:00',
            y0=0,
            x1='2020-10-25 08:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )


fig_SD_MAINTE_actual.update_shapes(dict(xref='x', yref='y'))

fig_SD_MAINTE_actual.add_annotation(
        x='2020-10-24 10:00:00',
        xref="x",
        yref="paper",
        text="Power Limitation",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#8e2800",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

fig_SD_MAINTE_actual.add_annotation(
        x='2020-11-09 05:00:00',
        xref="x",
        yref="paper",
        text="Power Limitation",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#8e2800",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

fig_SD_MAINTE_actual.add_annotation(
        x='2020-11-01 12:00:00',
        xref="x",
        yref="paper",
        text="E&I Blackout Activity",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#850017",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

In [ ]:
'#1f77b4',  // muted blue
'#ff7f0e',  // safety orange
'#2ca02c',  // cooked asparagus green
'#d62728',  // brick red

#### Export the chart as HTML

In [909]:
plt.close()
with open(r'C:\Users\v.t.flores\Documents\MS_Progress_of_Equipment_Handover_Tracker_actual_temporary.html', 'w') as f:
    f.write(fig_SD_MAINTE_actual.to_html(fig_SD_MAINTE_actual, full_html=True, include_plotlyjs='cdn'))


plt.close()
with open(r'\\THGOBNA002\thpal\05 Production\150 DCS\Miscellaneous\MS_Progress_of_Equipment_Handover_Tracker.html', 'w') as f:
    f.write(fig_SD_MAINTE_actual.to_html(fig_SD_MAINTE_actual, full_html=True, include_plotlyjs='cdn'))




In [908]:
plt.close()
with open(r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\ver\MS_Progress_of_Equipment_Handover_Tracker.html', 'w') as f:
    f.write(fig_SD_MAINTE_actual.to_html(fig_SD_MAINTE_actual, full_html=True, include_plotlyjs='cdn'))    

In [906]:
plt.close()
with open(r'C:\Users\v.t.flores\Documents\MS_Progress_of_Equipment_Handover_Tracker.html', 'w') as f:
    f.write(fig_SD_MAINTE_actual.to_html(fig_SD_MAINTE_actual, full_html=True, include_plotlyjs='cdn'))

### Annex codes

In [721]:
'''
# CODE BLOCK 6

df_pivot_list = []
for i in range(len(Reference_time_list)):
    df = df_Act_ref2[df_Act_ref2['Reference_time'] == Reference_time_list[i]]   ## get all the rows that match the same Reference_time
    df.columns = ['Activities','Reference_time', 'prcent_progress']
    df_i_pivot = df.pivot(columns='Reference_time', index='Activities', values='prcent_progress')
    df_pivot_list.append(df_i_pivot)
    
    # df_pivot_list will be used in the code below
    
'''
'''
# CODE BLOCK 7
#UPDATE THE ACTUAL_PROGRESS_UPDATE DATAFRAME using for loop and the df.at[row,col] = input_value
for df in df_pivot_list:
    for i in range(len(df)): #update code for updating a single reference datetime
        activity_name = df.iloc[i].name # this is the activity name
        ref_time = df.columns[0] # this is the column name
        progress_val = df.iloc[i]
        
        if ref_time in ACTUAL_PROGRESS_UPDATE.columns: #check if the ref time (same as the date) is in daterange

            ACTUAL_PROGRESS_UPDATE.at[activity_name, ref_time] = progress_val
            
        else:
            pass
print('PROGRESS Updated {}'.format(datetime.datetime.now()))            

# ACTUAL_PROGRESS_UPDATE
df_equipment_Grouped = ACTUAL_PROGRESS_UPDATE.groupby('Equipment')[ACTUAL_PROGRESS_UPDATE.columns[2:]].mean().T
'''

PROGRESS Updated 2020-10-16 14:33:17.258135


In [ ]:
# TEMPORARY CODE FOR GETTING THE ACTIVITIES PER EQUIPMENT PER AREA
"""
progress_plan_columns = shutdown_sched_4.columns[-504:] # the list of columns containing the %progress per activity as planned

List_equiments = shutdown_sched_4['Equipment'].unique() # list of the unique activities in the shutdown_sched_4 dataframe
List_equiments_rshp = List_equiments.reshape(len(List_equiments), 1) # array of all equipments shaped to fit in one column

# Initiallize the variables with zeros
SUM_percent_progress_of_activity_array = 0 #np.zeros((1, len(progress_plan_columns))) # the array containing all the %progress for a single equipment for the entire duration of the shutdown
SUM_percent_progress_of_equipment_array = np.zeros((len(List_equiments), len(progress_plan_columns))) # the array containing all the %progress for all the equipments for the entire duration of the shutdown


for eqpmnt_index in range(len(List_equiments_rshp)): #loop through the list of equipments
    eq = List_equiments_rshp[eqpmnt_index][0] # get a single name of the equipment from the list of equipments
    print(eq) # Print the equipment taken the list and check it in the output for confirmation
    count = shutdown_sched_4[(shutdown_sched_4['Equipment'] == eq)].count() # this counts the total number of activities under the same equipment. this includes the maintenance activities
    
    total_activities_count = 0 # initiallize the total_activities counter to zero. this counts all the activities prior to maintenance handover or inspection. so this counts the prodn activts leading up to handover
    percent_progress_of_activity_array = 0 # initiallize to zero. this will store an array of activity progress for the entire shutdown daterange
    SUM_percent_progress_of_activity_array = 0 # initiallize to zero. this will store an array of the percent progress of all the activities for the same equipment for the entire shutdown daterange
    for act in range(0, len(shutdown_sched_4[shutdown_sched_4['Equipment'] == eq])): # loop thru all the rows with the same equipment name eq
        
        activity_name = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['Activities'].iloc[act]   # get the activity name one by one
        percent_progress_of_activity_array = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq][progress_plan_columns].iloc[act]   # get the entire array of the progress for entire SD daterange
        # np.array() = activity_names
        # percent_progress_
        
        SUM_percent_progress_of_activity_array = SUM_percent_progress_of_activity_array + percent_progress_of_activity_array # add up all the progress of all the activities for the same equipment

        print(activity_name)
        match = re.search(r'\binspection\b',activity_name.lower()) # check the activity for the keyword 'inspection'
        match1 = re.search(r'\bmaintenance\b',activity_name.lower()) # check the activity for the keyword 'maintenance'
        if (not match) and (not match1) : # check if there is no match
            #print(activity_name)
            total_activities_count+=1
        else:
            print("index: {}, activitiy: {}".format(total_activities_count, activity_name))
            break # stop looping on the same equipment if the activity matches the keywords 'inspection' or 'maintenance'
    
    equipment_percent_progress_total = 100*total_activities_count
    SUM_percent_progress_of_equipment_array[eqpmnt_index] = 100*SUM_percent_progress_of_activity_array/equipment_percent_progress_total
       
df_prcnt_progress = pd.DataFrame(SUM_percent_progress_of_equipment_array)
df_prcnt_progress['equipment'] = List_equiments_rshp

"""
# END OF TEMPORARY CODE

In [ ]:
# Spare code
# DO NOT EXECUTE. This code block is for continuation
# This code block is non vectorized.
# For use in handling the input data for the actual progress which takes in the datetime when the activity is completed. DCS controller inputs this data manually.
#
#
"""
reference_datetime = parse('2020-10-31 02:00:00')#datetime.datetime.now()

# Initiallize the variables with zeros

equipment_percent_complte = np.zeros((len(List_equiments_rshp),1))

activities_arr = np.zeros((len(shutdown_sched_4['Activities']),1))
percent_prog_act_Plan_arr = np.zeros((len(shutdown_sched_4['Activities']),1))
reference_datetime_arr = np.zeros((len(shutdown_sched_4['Activities']),1))

for eqpmnt_index in range(len(List_equiments_rshp)):
    eq = List_equiments_rshp[eqpmnt_index][0]
    print(eq)
    count = shutdown_sched_4[(shutdown_sched_4['Equipment'] == eq) & (shutdown_sched_4['Complete_or_Not'] == 'Completed')].count()['Complete_or_Not']
    
    #total_act_per_eqpmnt = len(shutdown_sched_4[shutdown_sched_4['Equipment'] == eq])
    total_activities_count = 0
    for act in range(0, len(shutdown_sched_4[shutdown_sched_4['Equipment'] == eq])):
        
        activity_name = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['Activities'].iloc[act]
        time_start = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['START_plan'].iloc[act]
        duration = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['Duration_hrs_plan'].iloc[act]
        
        #Calculate the time difference in hours
        print("time start: {}, reference time: {}".format(time_start, reference_datetime))
        diff = reference_datetime - parse(str(time_start))
        diff_hours = diff.days*24 + diff.seconds/3600
        
        #(datetime_diff/duration).days/24 + (datetime_diff/duration).seconds/3600
        print(diff_hours)
        # Calculate the planned progress per activity
        #
        # 
        # Compute the planned shutdown progress
        if duration != 0: # check if the duration is not set to zero
        
            planned_progress_per_act = 100*diff_hours/duration
            if planned_progress_per_act > 0:
                if planned_progress_per_act >= 100:
                    planned_progress_per_act = 100
                    print("ref_time: {}, time start {}, activity: {}, %Progress {} %, duration:{}, index: {}".format(reference_datetime, time_start, activity_name, round(planned_progress_per_act, 2), duration, act))
                    percent_prog_act_Plan_arr[act] = round(planned_progress_per_act, 2) #store the calculated percent progress into the array called percent_prog_act_Plan_arr
                elif planned_progress_per_act < 100:
                    print("ref_time: {}, time start {}, activity: {}, %Progress {} %, duration:{}, index: {}".format(reference_datetime, time_start, activity_name, round(planned_progress_per_act, 2), duration, act))
                    percent_prog_act_Plan_arr[act] = round(planned_progress_per_act, 2) #store the calculated percent progress into the array called percent_prog_act_Plan_arr
                    
            else:
                planned_progress_per_act = 0
                print("ref_time: {}, time start {}, activity: {}, %Progress {} %, duration:{}, index: {}".format(reference_datetime, time_start, activity_name, round(planned_progress_per_act, 2), duration, act))
                percent_prog_act_Plan_arr[act] = planned_progress_per_act #store the calculated percent progress into the array called percent_prog_act_Plan_arr
        else:
            planned_progress_per_act = 0
            percent_prog_act_Plan_arr[act] = planned_progress_per_act #store the calculated percent progress into the array called percent_prog_act_Plan_arr
        #
        # End code for the percent progress calculation
        
        
        match = re.search(r'\binspection\b',activity_name.lower())
        if not match:
            print(activity_name)
            total_activities_count+=1
        else:
            #print("activity start: {}".format(shutdown_sched_4[['Activities','START_plan']].iloc[act]))
            break
    
    equipment_percent_progress = round(100*count/total_activities_count, 2)
    equipment_percent_complte[eqpmnt_index] = equipment_percent_progress
    #print(eq+ " count: {}. total_prg_to_H_over: {}, tot_act_count: {}".format(count, equipment_percent_progress, total_activities_count))
    
df_prcnt_progress = pd.DataFrame(equipment_percent_complte)
df_prcnt_progress['equipment'] = List_equiments_rshp
df_prcnt_progress = df_prcnt_progress.rename(columns={0:'Percent_Progress_to_HandOver', 'equipment':'Equipment_Tag'})
df_prcnt_progress = df_prcnt_progress[['Equipment_Tag','Percent_Progress_to_HandOver']]
"""


In [ ]:
# spare code
"""
diff = reference_datetime - parse(str(time_start))
diff_hours = diff.days*24 + diff.seconds/3600
print(diff_hours)
"""

In [ ]:
# Spare code
"""
df_Shutdown_date_range = pd.DataFrame(Shutdown_date_range) # Create dataframe from Shutdown_date_range array
df_prcnt_progress_transposed = df_prcnt_progress.T # Transpose the df_prcnt_progress dataframe
equiment_list = df_prcnt_progress_transposed.iloc[-1] # get the list of equipments by slicing the last row of the df_prcnt_progress_transposed dataframe
df_prcnt_progress_transposed = df_prcnt_progress_transposed.drop('equipment')  # drop the row with equipment
df_prcnt_progress_transposed['DATETIME'] = df_Shutdown_date_range[0] # Create a column called the DATETIME using the values from df_Shutdown_date_range
df_prcnt_progress_transposed = df_prcnt_progress_transposed.set_index('DATETIME') # set the DATETIME as the index
df_prcnt_progress_transposed.columns = equiment_list # rename the columns with the equipment list using the equipment_list variable


# replace the values in the df_prcnt_progress_transposed to a range 0 to 100
for col in range(len(df_prcnt_progress_transposed.columns)):
    column = df_prcnt_progress_transposed.columns[col]
    df_prcnt_progress_transposed[column] = np.where(df_prcnt_progress_transposed[column]>=100, 100, np.where(df_prcnt_progress_transposed[column]<0, 0, df_prcnt_progress_transposed[column]))

columns_dezn = ['De-Zn Feedline', '105TK01', '105TK02', '105TK03', '105TK05',
                '105SP01/02', '105PU01ABCDEFG', '105FT01A', '105FT01B', '105FT01C',
                '105FT01D', '105FT01E', '105FT01F', '105TK13', '105TK08']

columns_MS = ['106TK01', 'MS FEEDLINE', '106VE01', '106VE02', '106VE03', 
              '106VE04', '106VE05', '106HX02', '106VB01AB', '106CP01AB', 
              '106TH01', '106FT02A', '106FT02B', '106TK08', '106TK04',
              '106TK05', '106TK18', '106TK14', '106SP02', 'MS Common Vent/ 106SP01',
              '106SR00/02\n106SP03/04', '106SR03', '106SR04', '106TK15/ NaOH header line']

columns_108_chem = [ '108TK01', '108TK04','24" SPT tie in', 'VSLC RHL Line', '112TK00',
                    '112TK01', '114HP/FD01', '114TK01', '114TK02', '114 Pumps', 
                    '115TK01','115TK02']
columns_111_ls = ['111 Belt Conveyors\n111VS01\n111CR01AB', '111ML01','111TK01', '111PU01/\n111CY01', '111TK02', 
                  '530CT31', '525NG01']

MS_progress_plan = np.sum(df_prcnt_progress_transposed[columns_MS], axis=1)/len(columns_MS)
DeZn_progress_plan = np.sum(df_prcnt_progress_transposed[columns_dezn], axis=1)/len(columns_dezn)
FNTRL_chem_progress_plan = np.sum(df_prcnt_progress_transposed[columns_108_chem], axis=1)/len(columns_108_chem)
LS_CT_progress_plan = np.sum(df_prcnt_progress_transposed[columns_111_ls], axis=1)/len(columns_111_ls)

df_prcnt_progress_transposed['MS_progress_plan'] = MS_progress_plan
df_prcnt_progress_transposed['DeZn_progress_plan'] = DeZn_progress_plan
df_prcnt_progress_transposed['FNTRL_chem_progress_plan'] = FNTRL_chem_progress_plan
df_prcnt_progress_transposed['LS_CT_progress_plan'] = LS_CT_progress_plan
"""